<a href="https://colab.research.google.com/github/Dommex5/MY-FIRST-SQL-QUERY/blob/main/Copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load CSV files into DataFrames

customers= pd.read_csv('/olist_customers_dataset.csv')
orders= pd.read_csv('/olist_orders_dataset.csv')
products= pd.read_csv('/olist_products_dataset.csv')
order_payments= pd.read_csv('/olist_order_payments_dataset.csv')
order_reviews= pd.read_csv('/olist_order_reviews_dataset.csv')
order_items= pd.read_csv('/olist_order_items_dataset.csv')

print(customers.head())
# print(orders.head())
# print(Products.head())
# print(order_payments.head())
# print(order_reviews.head())
# print(order_items.head())

In [ ]:
# --- Transformations ---

# 1. Customer Dimension
customers_dim = customers[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']].copy()
customers_dim.columns = ['CustomerID', 'CustomerUniqueID', 'ZipCode', 'City', 'State']


# 2. Product Dimension
products_dim = products[['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].copy()
products_dim.columns = ['ProductID', 'CategoryName', 'ProductNameLength', 'ProductDescriptionLength', 'ProductPhotosQty', 'ProductWeightG', 'ProductLengthCm', 'ProductHeightCm', 'ProductWidthCm']

# 3. Product Category Dimension
product_categories_dim = products_dim[['CategoryName']].drop_duplicates().reset_index(drop=True)
product_categories_dim['CategoryID'] = product_categories_dim.index + 1
products_dim = products_dim.merge(product_categories_dim, on='CategoryName', how='left')

# 4. State Dimension
states_dim = customers_dim[['State']].drop_duplicates().reset_index(drop=True)
states_dim['StateID'] = states_dim.index + 1
customers_dim = customers_dim.merge(states_dim, on='State', how='left')


# 5. Date Dimension
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
dates_dim = pd.DataFrame({'Date': orders['order_purchase_timestamp'].dt.date.unique()})
dates_dim['OrderDateID'] = dates_dim.index + 1
dates_dim['Date'] = pd.to_datetime(dates_dim['Date'])
dates_dim['Day'] = dates_dim['Date'].dt.day
dates_dim['Month'] = dates_dim['Date'].dt.month
dates_dim['Year'] = dates_dim['Date'].dt.year
dates_dim['DayOfWeek'] = dates_dim['Date'].dt.day_name()

# 6. Time Dimension
times_dim = pd.DataFrame({'Time': orders['order_purchase_timestamp'].dt.time.unique()})
times_dim['OrderTimeID'] = times_dim.index + 1
times_dim['Hour'] = times_dim['Time'].apply(lambda x: x.hour)


# 7. Payment Type Dimension
payment_types_dim = order_payments[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_types_dim['PaymentTypeID'] = payment_types_dim.index + 1
payment_types_dim.columns = ['PaymentType', 'PaymentTypeID']

# 8. Sales Fact Table
# Convert 'order_id' to a unique index, handling potential duplicates
sales_fact =  products_dim.merge(customers_dim, left_on='product_id', right_on='CustomerID') \
                        .merge(products_dim, left_on='product_id', right_on='ProductID') \
                        .merge(dates_dim, left_on='order_purchase_timestamp', right_on='Date') \
                        .merge(times_dim, left_on='order_purchase_timestamp', right_on='Time') \
                        .merge(order_payments, left_on='order_id', right_on='order_id') \
                        .merge(payment_types_dim, on='payment_type')

sales_fact = sales_fact[['order_id', 'CustomerID', 'ProductID', 'OrderDateID', 'OrderTimeID', 'StateID', 'PaymentTypeID',
                         'order_item_id', 'price', 'freight_value', 'payment_sequential', 'payment_installments', 'payment_value']]

sales_fact.head()






KeyError: 'customer_id'